# Model Training Pipeline

## Overview

This notebook trains multiclass classification models to generate i-MAP scores for distinguishing between multiple genetic/phenotypic groups using high-content imaging data. The pipeline implements in-silico multiplexing by integrating cellular profiles from multiple marker combinations imaged across separate plates.

**Workflow:**
1. **Configuration**: Set analysis parameters including screen name, marker combinations, and genetic classification scheme
2. **Data Loading**: Load single-cell imaging features from multiple marker sets using the `ImageScreenMultiAntibody` class
3. **Preprocessing**: Filter and normalize features (remove missing/constant features, filter outlier cells, standardize measurements)
4. **Model Training**: Train deep learning classifiers using the MAP (Molecular ALS Phenotype) analysis framework with cross-validation strategies (e.g., leave-one-out, sample splits)
5. **Model Saving**: Serialize trained models and analysis results for downstream evaluation and scoring

**Key Components:**
- **Model Architecture**: Multi-marker deep learning classifier with hierarchical encoders (marker-specific → cell-level → sample-level)
- **Training Strategy**: Two-phase training with per-marker early stopping followed by integrated model optimization
- **Sample Splits**: Configurable cross-validation (leave-one-out or fixed train/test splits) ensuring unbiased predictions on held-out samples
- **Output**: Trained models with predicted class probabilities and feature importances saved to disk

The trained models can be used to score new samples or evaluate classification performance on held-out data.

## 1. Configuration
Set analysis parameters including the screen name, analysis type (e.g., multiclass), marker subset filter, and specific marker combinations to analyze.

In [ ]:
# ---- Parameters ----
SCREEN = "20250216_AWALS37_Full_screen_n96"
ANALYSIS = "multiclass_loocv"
ANTIBODY = ["HSP70/SOD1", "FUS/EEA1", "COX IV/Galectin3/atubulin"]
MARKER = "all"

Set random seeds across all libraries to ensure reproducible model training and evaluation.

In [ ]:
import os
import random
import numpy as np
import torch

# ---- Set seeds for reproducibility ----
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True, warn_only=True)

Load analysis parameters from JSON configuration file and update with notebook-specific settings (screen name, marker combinations, feature filters).

In [ ]:
from maps.screens import ImageScreenMultiAntibody
import json
from pathlib import Path


# --- Initialize parameters ---
pdir = Path("/home/kkumbier/maps/template_analyses/pipelines/params")
with open(pdir / f"{ANALYSIS}.json", "r") as f:
    params = json.load(f)

params["screen"] = SCREEN
params["antibodies"] = ANTIBODY

# Update marker if specified (e.g., to drop specific markers)
if MARKER != "all":
    fstr = params["preprocess"]["drop_feature_types"]["feature_str"]
    fstr += f"|^.*{MARKER}.*$"
    params["preprocess"]["drop_feature_types"]["feature_str"] = fstr

## 2. Data Loading & 3. Preprocessing
Load single-cell imaging features from multiple marker combinations and apply preprocessing steps: feature filtering, outlier removal, missing value imputation, and normalization.

In [4]:
# Load and process screens for train / test
screen = ImageScreenMultiAntibody(params)
screen.load(antibody=params["antibodies"])

print("Processing data...")
screen.preprocess()
assert screen.data is not None, "Loading failed"

for ab in params["antibodies"]:
    print(f"Marker set: {ab}")
    print(f"Data: {screen.data[ab].shape}")

Processing data...
Preprocessing complete
Marker set: FUS/EEA1
Data: (87657, 315)
Marker set: COX IV/Galectin3/atubulin
Data: (83045, 312)
Marker set: HSP70/SOD1
Data: (86124, 311)


## 4. Model Training
Train multiclass classification models using the MAP analysis framework. The `fit()` method executes the complete training pipeline including cross-validation, model fitting, and prediction generation on held-out samples.

In [ ]:
from maps.analyses import MAP
map_analysis = MAP(screen)
map_analysis.fit()

---Training hold-out: ALS32---
Starting cell-level training...
Cell Epoch 1/100, Overall Loss: 1.1809, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.1884, Acc: 0.3346 
  FUS/EEA1 - Loss: 1.1542, Acc: 0.3648 
  HSP70/SOD1 - Loss: 1.2000, Acc: 0.3485 


Cell Epoch 2/100, Overall Loss: 1.1251, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.1188, Acc: 0.4031 
  FUS/EEA1 - Loss: 1.1041, Acc: 0.4075 
  HSP70/SOD1 - Loss: 1.1524, Acc: 0.3685 
Cell Epoch 3/100, Overall Loss: 1.1121, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.1218, Acc: 0.4019 
  FUS/EEA1 - Loss: 1.0619, Acc: 0.4450 
  HSP70/SOD1 - Loss: 1.1528, Acc: 0.3631 
Cell Epoch 4/100, Overall Loss: 1.0998, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.1100, Acc: 0.4042 
  FUS/EEA1 - Loss: 1.0561, Acc: 0.4581 
  HSP70/SOD1 - Loss: 1.1333, Acc: 0.3756 
Cell Epoch 5/100, Overall Loss: 1.0975, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.0914, Acc: 0.4262 
  FUS/EEA1 - Loss: 1.0715, Acc: 0.4467 
  HSP70/SOD1 - Loss: 1.1296, Acc: 0.3723 
Cell Epoch 6/100, Overall Loss: 1.0866, Active: 3/3
  COX IV/Galectin3/atubulin - Loss: 1.1055, Acc: 0.4044 
  FUS/EEA1 - Loss: 1.0434, Acc: 0.4629 
  HSP70/SOD1 - Loss: 1.1108, Acc: 0.3944 
Cell Epoch 7/100, Overall Loss: 1.0620, Activ

/awlab/projects/2024_ALS/software/miniforge3/envs/maps/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Cell Epoch 17/100, Overall Loss: 1.0319, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 0.9848, Acc: 0.5219 
  HSP70/SOD1 - Loss: 1.0790, Acc: 0.4167 
Cell Epoch 18/100, Overall Loss: 1.0199, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 0.9669, Acc: 0.5383 
  HSP70/SOD1 - Loss: 1.0729, Acc: 0.4304 
Cell Epoch 19/100, Overall Loss: 1.0460, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 1.0023, Acc: 0.5125 
  HSP70/SOD1 - Loss: 1.0897, Acc: 0.4094 
Cell Epoch 20/100, Overall Loss: 1.0243, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 0.9876, Acc: 0.5115 
  HSP70/SOD1 - Loss: 1.0609, Acc: 0.4396 
Cell Epoch 21/100, Overall Loss: 1.0114, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 0.9648, Acc: 0.5358 
  HSP70/SOD1 - Loss: 1.0580, Acc: 0.4512 


/awlab/projects/2024_ALS/software/miniforge3/envs/maps/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


...

Cell Epoch 16/100, Overall Loss: 1.0212, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 0.9841, Acc: 0.5181 
  HSP70/SOD1 - Loss: 1.0582, Acc: 0.4467 
  Freezing HSP70/SOD1 encoder at epoch 17
Cell Epoch 17/100, Overall Loss: 1.0749, Active: 1/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 1.0483, Acc: 0.4560 
  HSP70/SOD1 - Loss: 1.1015, Acc: 0.3919 [FROZEN]
Cell Epoch 18/100, Overall Loss: 1.0044, Active: 1/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 1.0044, Acc: 0.5067 
  HSP70/SOD1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
Cell Epoch 19/100, Overall Loss: 1.0370, Active: 1/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Loss: 1.0370, Acc: 0.4750 
  HSP70/SOD1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
Cell Epoch 20/100, Overall Loss: 1.0099, Active: 1/3
  COX IV/Galectin3/atubulin - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  FUS/EEA1 - Los

/awlab/projects/2024_ALS/software/miniforge3/envs/maps/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Cell Epoch 15/100, Overall Loss: 1.0729, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 1.0683, Acc: 0.4235 
  FUS/EEA1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  HSP70/SOD1 - Loss: 1.0775, Acc: 0.4292 
Cell Epoch 16/100, Overall Loss: 1.0634, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 1.0538, Acc: 0.4446 
  FUS/EEA1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  HSP70/SOD1 - Loss: 1.0731, Acc: 0.4369 
  Freezing HSP70/SOD1 encoder at epoch 17
Cell Epoch 17/100, Overall Loss: 1.0581, Active: 1/3
  COX IV/Galectin3/atubulin - Loss: 1.0537, Acc: 0.4352 
  FUS/EEA1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  HSP70/SOD1 - Loss: 1.0625, Acc: 0.4475 [FROZEN]
Cell Epoch 18/100, Overall Loss: 1.0611, Active: 1/3
  COX IV/Galectin3/atubulin - Loss: 1.0611, Acc: 0.4440 
  FUS/EEA1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
  HSP70/SOD1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
Cell Epoch 19/100, Overall Loss: 1.0656, Active: 1/3
  COX IV/Galectin3/atubulin - Loss: 1.0656, Acc: 0.4356 
  FUS/EEA1 - Loss: 0.0000, Acc: 

/awlab/projects/2024_ALS/software/miniforge3/envs/maps/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Cell Epoch 17/100, Overall Loss: 1.0459, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 1.0485, Acc: 0.4535 
  FUS/EEA1 - Loss: 1.0432, Acc: 0.4635 
  HSP70/SOD1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
Cell Epoch 18/100, Overall Loss: 1.0243, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 1.0436, Acc: 0.4669 
  FUS/EEA1 - Loss: 1.0051, Acc: 0.4977 
  HSP70/SOD1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
Cell Epoch 19/100, Overall Loss: 1.0358, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 1.0605, Acc: 0.4581 
  FUS/EEA1 - Loss: 1.0111, Acc: 0.4890 
  HSP70/SOD1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
Cell Epoch 20/100, Overall Loss: 1.0575, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 1.0928, Acc: 0.4185 
  FUS/EEA1 - Loss: 1.0221, Acc: 0.4894 
  HSP70/SOD1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]
Cell Epoch 21/100, Overall Loss: 1.0196, Active: 2/3
  COX IV/Galectin3/atubulin - Loss: 1.0510, Acc: 0.4619 
  FUS/EEA1 - Loss: 0.9882, Acc: 0.5102 
  HSP70/SOD1 - Loss: 0.0000, Acc: 0.0000 [FROZEN]


## 5. Model Saving
Serialize the trained MAP analysis object (including fitted models, predictions, and feature importances) along with configuration parameters to disk for downstream evaluation.

In [ ]:
import pickle

ab_string = "_".join(ANTIBODY).replace("/", "-")
output_dir = Path(params.get("result_dir")) / params.get("screen")
with open(output_dir / f"{ANALYSIS}-{ab_string}-{MARKER}.pkl", "wb") as f:
    pickle.dump({"analysis": map_analysis, "params": params}, f)